In [3]:
import cobra  #porque lo estoy importando
import pandas as pd
import escher
%matplotlib inline
from cobra import Reaction, Metabolite
import pandas as pd
pd.options.display.max_rows = 30
from cameo.flux_analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pandas import DataFrame
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from IPython.display import display
import re
import gurobipy
from cobra.io import read_sbml_model
from cameo.strain_design import pathway_prediction
from cameo.models import universal
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo.strain_design.deterministic.linear_programming import OptKnock
from cobra.io import write_sbml_model

In [4]:
modelCore = cobra.io.load_json_model('../models/e_coli_core.json')
display(modelCore)
modeliML = cobra.io.load_json_model('../models/iML1515.json')
display(modeliML)

Name,e_coli_core
Memory address,0x07f888ca27eb0
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


Name,iML1515
Memory address,0x07f8874e98490
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
metReactions = [
    {
        'id':'ACLS',
        'Name': 'Acetolactate synthase',
        'Metabolites': {
            'h_c':-1,
            'pyr_c': -2,
            'alac__S_c':1,
            'co2_c': 1
        },
        'gene_reaction_rule':'(ilvG or ilvH)'
    },
    {
        'id':'ACLDC',
        'Name': 'Acetolactate decarboxylase',
        'Metabolites': {
            'alac__S_c':-1,
            'h_c': -1,
            'actn__R_c':1,
            'co2_c': 1
        },
        'gene_reaction_rule':'(budA)'
        
    }, 
    {
        'id':'BTD_RR',
        'Name': 'R R butanediol dehydrogenase',
        'Metabolites': {
            'btd_RR_c':1,
            'nad_c': 1,
            'actn__R_c':-1,
            'h_c': -1, 
            'nadh_c':-1
        },
        'gene_reaction_rule':'(adh or bdhA)'
    }, 
    {
        'id':'BTDt_RR', 
        'Name': 'R R  butanediol transport',
        'Metabolites': {
            'btd_RR_c':-1,
            'btd_RR_e': 1
        },
        'gene_reaction_rule':''
    },
    {
        'id':'EX_btd_RR_e', 
        'Name': 'R R  2 3 Butanediol exchange',
        'Metabolites': {
            'btd_RR_e':-1
        },
        'gene_reaction_rule':''
        
    }
]

In [6]:
def check_reactions(model, metReactions):
    rxExist = []
    rxNexist = []
    for rx in metReactions:
        try: 
            model.reactions.get_by_id(rx['id'])
            rxExist.append(rx)
        except KeyError as e: 
            rxNexist.append(rx)

    print('Las reacciones que están:', rxExist)
    print('Las reacciones que no están:', rxNexist)
    
    return rxExist, rxNexist

In [7]:
rxExistCore, rxNexistCore = check_reactions(modelCore,metReactions)
rxExistiML, rxNexistiML = check_reactions(modeliML,metReactions)

Las reacciones que están: []
Las reacciones que no están: [{'id': 'ACLS', 'Name': 'Acetolactate synthase', 'Metabolites': {'h_c': -1, 'pyr_c': -2, 'alac__S_c': 1, 'co2_c': 1}, 'gene_reaction_rule': '(ilvG or ilvH)'}, {'id': 'ACLDC', 'Name': 'Acetolactate decarboxylase', 'Metabolites': {'alac__S_c': -1, 'h_c': -1, 'actn__R_c': 1, 'co2_c': 1}, 'gene_reaction_rule': '(budA)'}, {'id': 'BTD_RR', 'Name': 'R R butanediol dehydrogenase', 'Metabolites': {'btd_RR_c': 1, 'nad_c': 1, 'actn__R_c': -1, 'h_c': -1, 'nadh_c': -1}, 'gene_reaction_rule': '(adh or bdhA)'}, {'id': 'BTDt_RR', 'Name': 'R R  butanediol transport', 'Metabolites': {'btd_RR_c': -1, 'btd_RR_e': 1}, 'gene_reaction_rule': ''}, {'id': 'EX_btd_RR_e', 'Name': 'R R  2 3 Butanediol exchange', 'Metabolites': {'btd_RR_e': -1}, 'gene_reaction_rule': ''}]
Las reacciones que están: [{'id': 'ACLS', 'Name': 'Acetolactate synthase', 'Metabolites': {'h_c': -1, 'pyr_c': -2, 'alac__S_c': 1, 'co2_c': 1}, 'gene_reaction_rule': '(ilvG or ilvH)'}]
Las

In [8]:
metMetabolites = [
    {
        'id': 'h_c',
        'formula': 'H',
        'name': 'H+',
        'compartment': 'c'
    },
    {
        'id': 'pyr_c',
        'formula': 'C3H3O3',
        'name': 'Pyruvate',
        'compartment': 'c'
    },
    {
        'id': 'alac__S_c',
        'formula': 'C5H8O4',
        'name': '(S)-2-Acetolactate',
        'compartment': 'c'
    },
    {
        'id': 'co2_c',
        'formula': 'CO2',
        'name': 'CO2 CO2',
        'compartment': 'c'
    },
    {
        'id': 'actn__R_c',
        'formula': 'C4H8O2',
        'name': 'R  Acetoin',
        'compartment': 'c'
    },
    {
        'id': 'btd_RR_c',
        'formula': 'C4H10O2',
        'name': 'R R  2 3 Butanediol',
        'compartment': 'c'
    },
    {
        'id': 'nadh_c',
        'formula': 'C21H27N7O14P2',
        'name': 'Nicotinamide adenine dinucleotide - reduced',
        'compartment': 'c'
    },
    {
        'id': 'nad_c',
        'formula': 'C21H26N7O14P2',
        'name': 'Nicotinamide adenine dinucleotide',
        'compartment': 'c'
    },
    {
        'id': 'btd_RR_e',
        'formula': 'C4H10O2',
        'name': 'R R  2 3 Butanediol',
        'compartment': 'e'
    }
]

In [9]:
def check_metabolites(model, metMetabolites):
    metExist = []
    metNexist = []

    for met in metMetabolites:
        try: 
            model.metabolites.get_by_id(met['id'])
            metExist.append(met)
        except KeyError as e: 
            metNexist.append(met)

    print('Los metabolitos que están:', metExist)
    print('Los metabolitos que no están:', metNexist) 
    return metExist, metNexist

In [10]:
metExistCore, metNexistCore = check_metabolites(modelCore, metMetabolites)
metExistiML, metNexistiML = check_metabolites(modeliML, metMetabolites)

Los metabolitos que están: [{'id': 'h_c', 'formula': 'H', 'name': 'H+', 'compartment': 'c'}, {'id': 'pyr_c', 'formula': 'C3H3O3', 'name': 'Pyruvate', 'compartment': 'c'}, {'id': 'co2_c', 'formula': 'CO2', 'name': 'CO2 CO2', 'compartment': 'c'}, {'id': 'nadh_c', 'formula': 'C21H27N7O14P2', 'name': 'Nicotinamide adenine dinucleotide - reduced', 'compartment': 'c'}, {'id': 'nad_c', 'formula': 'C21H26N7O14P2', 'name': 'Nicotinamide adenine dinucleotide', 'compartment': 'c'}]
Los metabolitos que no están: [{'id': 'alac__S_c', 'formula': 'C5H8O4', 'name': '(S)-2-Acetolactate', 'compartment': 'c'}, {'id': 'actn__R_c', 'formula': 'C4H8O2', 'name': 'R  Acetoin', 'compartment': 'c'}, {'id': 'btd_RR_c', 'formula': 'C4H10O2', 'name': 'R R  2 3 Butanediol', 'compartment': 'c'}, {'id': 'btd_RR_e', 'formula': 'C4H10O2', 'name': 'R R  2 3 Butanediol', 'compartment': 'e'}]
Los metabolitos que están: [{'id': 'h_c', 'formula': 'H', 'name': 'H+', 'compartment': 'c'}, {'id': 'pyr_c', 'formula': 'C3H3O3', '

In [11]:
def metabolites_to_add(metNexist):
    met_to_add = []
    for met in metNexist:
        m = Metabolite(id=met['id'], formula=met['formula'], name=met['name'], compartment=met['compartment'])
        met_to_add.append(m)
    print(met_to_add)
    return met_to_add

In [12]:
met_to_add_core = metabolites_to_add(metNexistCore)
met_to_add_iML = metabolites_to_add(metNexistiML)

[<Metabolite alac__S_c at 0x7f8870816250>, <Metabolite actn__R_c at 0x7f88708169a0>, <Metabolite btd_RR_c at 0x7f8870816c70>, <Metabolite btd_RR_e at 0x7f8870816bb0>]
[<Metabolite actn__R_c at 0x7f88708165e0>, <Metabolite btd_RR_c at 0x7f8870816640>, <Metabolite btd_RR_e at 0x7f8870816850>]


In [13]:
modelCore.add_metabolites(met_to_add_core)
modeliML.add_metabolites(met_to_add_iML)

In [14]:
def reactions_to_add(model, rxNexist):
    rxs_to_add = []
    for rx in rxNexist:
        x = Reaction(id=rx['id'], name=rx['Name'], lower_bound=-1000.0)
        if(len(rx['gene_reaction_rule'])>0):
            print('esto es:', rx['gene_reaction_rule'])
            x.gene_reaction_rule = rx['gene_reaction_rule']
        rxs_to_add.append(x)
        model.add_reactions([x])
        x.add_metabolites(rx['Metabolites'])
    print('reactions added: ',rxs_to_add)
    print('\n')

In [15]:
reactions_to_add(modelCore, rxNexistCore)
reactions_to_add(modeliML, rxNexistiML)

esto es: (ilvG or ilvH)
esto es: (budA)
esto es: (adh or bdhA)
reactions added:  [<Reaction ACLS at 0x7f88848f7a00>, <Reaction ACLDC at 0x7f88848f7040>, <Reaction BTD_RR at 0x7f8870816a30>, <Reaction BTDt_RR at 0x7f887051ffd0>, <Reaction EX_btd_RR_e at 0x7f8876745be0>]


esto es: (budA)
esto es: (adh or bdhA)
reactions added:  [<Reaction ACLDC at 0x7f88848f7880>, <Reaction BTD_RR at 0x7f88848f7850>, <Reaction BTDt_RR at 0x7f88848f76d0>, <Reaction EX_btd_RR_e at 0x7f88848f7370>]




In [20]:
modelCore.reactions.EX_btd_RR_e.lower_bound = 0
modelCore.reactions.ACLS.lower_bound = 0
modelCore.reactions.ACLDC.lower_bound = 0
modelCore.reactions.BTD_RR.lower_bound = 0
modelCore.reactions.BTDt_RR.lower_bound = 0

modeliML.reactions.EX_btd_RR_e.lower_bound = 0
modeliML.reactions.ACLS.lower_bound = 0
modeliML.reactions.ACLDC.lower_bound = 0
modeliML.reactions.BTD_RR.lower_bound = 0
modeliML.reactions.BTDt_RR.lower_bound = 0

In [22]:
def modify_medium(model,met, value):
    modelA = model.copy()
    medium = model.medium
    medium[met] = value
    modelA.medium = medium
    return modelA

In [23]:
modeliMLA = modify_medium(modeliML, 'EX_o2_e', 0)
modelCoreA = modify_medium(modelCore, 'EX_o2_e', 0)

Read LP format model from file /var/folders/v5/zrtrnkbn4vz0f42d5z7lj2jh0000gn/T/tmpgausvjoh.lp
Reading time = 0.05 seconds
: 1880 rows, 5432 columns, 21174 nonzeros
Read LP format model from file /var/folders/v5/zrtrnkbn4vz0f42d5z7lj2jh0000gn/T/tmp9f67vcek.lp
Reading time = 0.00 seconds
: 76 rows, 200 columns, 752 nonzeros


In [24]:
write_sbml_model(modelCore, "models/modelCore.xml")
write_sbml_model(modelCoreA, "models/modelCoreA.xml")
write_sbml_model(modeliML, "models/modeliML.xml")
write_sbml_model(modeliMLA, "models/modeliMLA.xml")

In [25]:
display(modelCore)
display(modelCoreA)
display(modeliML)
display(modeliMLA)

Name,e_coli_core
Memory address,0x07f888ca27eb0
Number of metabolites,76
Number of reactions,100
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


Name,e_coli_core
Memory address,0x07f887733e820
Number of metabolites,76
Number of reactions,100
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


Name,iML1515
Memory address,0x07f8874e98490
Number of metabolites,1880
Number of reactions,2716
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


Name,iML1515
Memory address,0x07f887733ebe0
Number of metabolites,1880
Number of reactions,2716
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
metReactions = [
    {
        'id':'id de la reacción segun BiGG',
        'Name': 'Nombre reacción',
        'Metabolites': {
            'A':-1,
            'B': -2,
            'C':1,
            'D': 1
        },
    }]